## Download MS-based data - buildings and streets

Define functions to download the raw data from Microsoft blob storage. If the data is not available it might work to replace *.blob.* with *.z5.web.* in all urls. There should be no difference in the data. 

Note: At the bottom of the notebook i try to download the same data from planatery computer and its 1:1, but there are still missing buildings in for example the centre of prague, which are there in the microsoft planetary computer explorer.

In [5]:
import geopandas as gpd
import pandas as pd

In [6]:
countries = ['Czechia', 'CzechRepublic', 'Poland', 'Germany', 'Austria', 'Slovakia']

In [11]:
def read_ms_buildings(url):
    """Read the data from the blob url, convert to a geodataframe and change to european planar coordinates"""
    from shapely.geometry import shape
    df = pd.read_json(url, lines=True)
    df['geometry'] = df['geometry'].apply(shape)
    gdf = gpd.GeoDataFrame(df, crs=4326).to_crs(epsg=3035)
    return gdf

def read_microsoft_country_data(country):
    """Search through the microsoft list and read all the buildings for a particular country"""
    dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
    location_links = dataset_links[dataset_links.Location == country]

    from joblib import Parallel, delayed    
    n_jobs = -1
    all_data = Parallel(n_jobs=n_jobs)(
        delayed(read_ms_buildings)(row.Url) for _, row in location_links.iterrows()
    )

    return pd.concat(all_data, ignore_index=True)


In [38]:
for country in countries:
    print('processing', country)
    gdf = read_microsoft_country_data(country)
    gdf = gdf.sort_values('geometry').reset_index(drop=True)
    gdf.to_parquet(f'/data/uscuni-eurofab/microsoft_buildings/ms_{country.lower()}.pq', 
                   geometry_encoding='geoarrow', 
                   write_covering_bbox=True, schema_version='1.1.0')

processing CzechRepublic


Merge all data together for regional deliniations

In [5]:
all_gdf = []
for country in countries:
    all_gdf.append(gpd.read_parquet(f'/data/uscuni-eurofab/microsoft_buildings/ms_{country.lower()}.pq'))
all_gdf = pd.concat(all_gdf, ignore_index=True)

In [6]:
all_gdf.shape

(56845150, 3)

In [7]:
all_gdf.head()

,type,properties,geometry
0,Feature,"{'confidence': 0.966099977493286, 'height': 1....","POLYGON ((4555515.661 2895209.536, 4555524.361..."
1,Feature,"{'confidence': 0.9919000267982481, 'height': 0...","POLYGON ((4555515.825 2895226.843, 4555513.436..."
2,Feature,"{'confidence': 0.9606999754905701, 'height': 4...","POLYGON ((4555531.043 2895386.444, 4555534.294..."
3,Feature,"{'confidence': 0.981299996376037, 'height': 3....","POLYGON ((4556673.328 2895445.262, 4556667.32 ..."
4,Feature,"{'confidence': 0.9782999753952021, 'height': 1...","POLYGON ((4556711.641 2895210.336, 4556717.7 2..."


In [8]:
all_gdf.to_parquet(f'/data/uscuni-eurofab/microsoft_buildings/ms_ce.pq')

## Define regions on MS data

Split the whole Central Europe to sub-regions based on their morphological continuity.

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
gdf = gpd.read_parquet('/data/uscuni-eurofab/microsoft_buildings/ms_ce.pq')

Aggregate the building centroids to 100x100m grid and use count as weight for DBSCAN.

In [3]:
cents = gdf.centroid
gdf['x'], gdf['y'] = cents.x, cents.y
gdf['id'] = gdf.index.values
data = gdf[["x", "y", 'id']]

In [4]:
data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)
grid = data[["id", "x_100", "y_100"]].groupby(["x_100", "y_100"]).count().reset_index()

/tmp/ipykernel_21827/4289293720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)
/tmp/ipykernel_21827/4289293720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["x_100", "y_100"]] = np.around(data[["x", "y"]], decimals=-2)


Cluster the grid cells in DBSCAN with 400 meters distance. Equivalent to  400- ~550 distance between buildings.

In [5]:
dbscan = DBSCAN(400, n_jobs=-1).fit(grid[["x_100", "y_100"]], sample_weight=grid["id"])
grid["labels"] = dbscan.labels_

In [6]:
data = pd.merge(data, grid, "left", on=["x_100", "y_100"])

Identify core urban areas (with more than 10k buildings within a single cluster).

In [7]:
counts = data.labels.value_counts()
data["core"] = data.labels.isin(counts[counts > 10000].index.drop(-1))

In [8]:
cores = data[data.core]

Assign each non-core cluster and non-core building without a cluster to the nearest cluster. Done using the grid for efficiency.

In [9]:
grid["core"] = grid.labels.isin(counts[counts > 10000].index.drop(-1))
grid_cores = grid[grid.core]
grid_cores = gpd.GeoDataFrame(
    grid_cores["labels"],
    geometry=gpd.points_from_xy(grid_cores["x_100"], grid_cores["y_100"]),
    crs=3035,
)
grid_cores_dissolved = grid_cores.dissolve("labels")

In [10]:
grid_non_cores = grid[~grid.core]
grid_non_cores = gpd.GeoDataFrame(
    grid_non_cores["labels"],
    geometry=gpd.points_from_xy(grid_non_cores["x_100"], grid_non_cores["y_100"]),
    crs=3035,
)

In [11]:
grid_non_cores_clustered = grid_non_cores[grid_non_cores.labels != -1]
grid_non_cores_outliers = grid_non_cores[grid_non_cores.labels == -1]

In [12]:
grid_non_cores_clustered_dissolved = grid_non_cores_clustered.dissolve("labels")

In [13]:
%%time
nearest = grid_cores.sindex.nearest(
    grid_non_cores_clustered_dissolved.geometry, return_all=False
)

CPU times: user 30 s, sys: 146 ms, total: 30.1 s
Wall time: 30.1 s


In [14]:
grid_non_cores_clustered_dissolved["nearest_core"] = grid_cores.labels.values[
    nearest[1]
]

In [15]:
%%time
nearest_outliers = grid_cores.sindex.nearest(
    grid_non_cores_outliers.geometry, return_all=False
)

CPU times: user 4.18 s, sys: 972 μs, total: 4.19 s
Wall time: 4.19 s


Finalise and Create final region label.

In [16]:
grid_non_cores_outliers["nearest_core"] = grid_cores.labels.values[nearest_outliers[1]]

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [17]:
grid_non_cores = pd.concat(
    [
        grid_non_cores_clustered_dissolved.reset_index().explode(ignore_index=True),
        grid_non_cores_outliers,
    ],
    ignore_index=True,
)

In [18]:
grid_non_cores["x_100"] = grid_non_cores.geometry.x
grid_non_cores["y_100"] = grid_non_cores.geometry.y

In [19]:
data = pd.merge(
    data,
    grid_non_cores[["x_100", "y_100", "nearest_core"]],
    "left",
    on=["x_100", "y_100"],
)

In [20]:
data["region"] = data.labels
data.loc[~data.core, "region"] = data.loc[~data.core, "nearest_core"]

In [21]:
data = data.rename(
    columns={
        "id_x": "id",
        "id_y": "weight",
        "labels": "dbscan_cluster",
    }
)

In [22]:
data

,x,y,id,x_100,y_100,weight,dbscan_cluster,core,nearest_core,region
0,4.555524e+06,2.895213e+06,0,4555500.0,2895200.0,2,57506,False,51525.0,51525
1,4.555513e+06,2.895226e+06,1,4555500.0,2895200.0,2,57506,False,51525.0,51525
2,4.555531e+06,2.895390e+06,2,4555500.0,2895400.0,3,57506,False,51525.0,51525
3,4.556668e+06,2.895443e+06,3,4556700.0,2895400.0,2,58093,False,51525.0,51525
4,4.556712e+06,2.895217e+06,4,4556700.0,2895200.0,1,58093,False,51525.0,51525
...,...,...,...,...,...,...,...,...,...,...
56845145,5.082977e+06,2.850046e+06,56845145,5083000.0,2850000.0,13,120741,False,114334.0,114334
56845146,5.083101e+06,2.850070e+06,56845146,5083100.0,2850100.0,4,120741,False,114334.0,114334
56845147,5.083083e+06,2.850104e+06,56845147,5083100.0,2850100.0,4,120741,False,114334.0,114334
56845148,5.083083e+06,2.850120e+06,56845148,5083100.0,2850100.0,4,120741,False,114334.0,114334


Save data

In [23]:
pd.concat(
    [
        grid_cores,
        grid_non_cores[["nearest_core", "geometry"]].rename(
            columns={"nearest_core": "labels"}
        ),
    ]
).dissolve("labels").convex_hull.to_frame("convex_hull").to_parquet(
    "/data/uscuni-eurofab/regions/ms_ce_region_hulls.parquet"
) 

In [24]:
data.to_parquet("/data/uscuni-eurofab/regions/ms_id_to_region.parquet")

Split all buildings into regions.

In [25]:
%%time
for region_id, group in data.groupby('region'):

    region_id = int(region_id)
    
    buildings = gdf.iloc[group.id]
    buildings['iid'] = buildings.index.values
    buildings.to_parquet(f'/data/uscuni-eurofab/regions/buildings/buildings_{region_id}.pq')

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value

CPU times: user 1min 2s, sys: 9.9 s, total: 1min 12s
Wall time: 1min 13s


In [1]:
# gpd.read_parquet("/data/uscuni-eurofab/regions/ms_ce_region_hulls.parquet").explore()

## Download overture streets

Download and store overture streets, based on the generated regions.

In [1]:
from core.generate_streets import read_overture_region_streets

import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [3]:
streets_dir = '/data/uscuni-eurofab/overture_streets/'
regions_datadir = "/data/uscuni-eurofab/"

In [4]:
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "ms_ce_region_hulls.parquet"
    )
region_hulls.shape

(474, 1)

In [7]:
def process_and_save(region_hull, region_id):
    print('Processing', region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

In [8]:
downloaded_streets = glob.glob(streets_dir + '*')
downloaded_streets = [int(s.split('_')[-1][:-3]) for s in downloaded_streets]

In [9]:
region_hulls = region_hulls.loc[~region_hulls.index.isin(downloaded_streets), ]
region_hulls

,convex_hull
labels,
19,"POLYGON ((4051500 3017500, 4049500 3017700, 40..."
24,"POLYGON ((4034200 3104600, 4033800 3105300, 40..."
33,"POLYGON ((4097600 3030400, 4054700 3105100, 40..."
478,"POLYGON ((4061200 3036100, 4059200 3052500, 40..."
754,"POLYGON ((4060900 2943800, 4059600 2944400, 40..."
...,...
133611,"POLYGON ((5240200 3128600, 5220200 3145300, 52..."
133765,"POLYGON ((5249800 3174800, 5244600 3175200, 52..."
133801,"POLYGON ((5256000 3068600, 5243700 3077600, 52..."


In [ ]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id) for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)


In [5]:
%%time

all_streets = []
for region_id, _ in region_hulls.to_crs(epsg=4326).iterrows():
    streets = gpd.read_parquet(streets_dir + f'streets_{region_id}.pq')
    all_streets.append(streets)
all_streets = pd.concat(all_streets)

CPU times: user 2min 8s, sys: 38.8 s, total: 2min 46s
Wall time: 2min 5s


In [8]:
all_streets.drop_duplicates('id').shape

(23332865, 21)

#### Planatery computer cache (this data is not used, but MS has a dump of old versions on the stack API)

In [1]:
import planetary_computer
import pystac_client
import geopandas as gpd
import pandas as pd

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

In [3]:
asset = collection.assets["delta"]
storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}

In [4]:
regions_datadir = "/data/uscuni-eurofab/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "ms_ce_region_hulls.parquet"
    )
region_hulls.shape

(474, 1)

In [7]:
search = catalog.search(
    collections=["ms-buildings"],
     bbox=region_hulls.to_crs(epsg=4326).total_bounds,
)

ic = search.item_collection()
len(ic)

824

In [8]:
import adlfs

fs = adlfs.AzureBlobFileSystem(
    **ic[0].assets["data"].extra_fields["table:storage_options"]
)

prefixes = [item.assets["data"].href for item in ic]
parts = []
for item in ic:
    parts.extend(fs.ls(item.assets["data"].href))

len(parts)

5307

In [12]:
%%time
res = []
for i, part in enumerate(parts):
    print(part)
    data = gpd.read_parquet(f"az://{part}", storage_options=storage_options)
    data.to_parquet(f'../data/ms_buildings_stack_part_{i}.pq')

footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=Switzerland/quadkey=120221210/part-00339-c0935363-1dc8-4d37-8df8-4a7301e2737b.c000.snappy.parquet
footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=Switzerland/quadkey=120221203/part-00623-350c3a8f-7c34-431a-a7e8-88cb8efa3d1d.c000.snappy.parquet
footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=Switzerland/quadkey=120221202/part-00435-a67f31b9-8dbd-4ffc-8d63-53c5a3417dfc.c000.snappy.parquet
footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=Switzerland/quadkey=120221201/part-00490-d15e5416-c11b-401d-9d71-2f8356edfe19.c000.snappy.parquet
footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=Switzerland/quadkey=120221200/part-00604-c5388fc8-e3f3-4486-9bd6-3d03e2d07b31.c000.snappy.parquet
footprints/delta/2023-04-25/ml-buildings.parquet/RegionName=Switzerland/quadkey=120221122/part-00039-190ea88e-2c8f-4c86-8fbd-bf9acb8e831f.c000.snappy.parquet
footprints/delta/2023-04-25/ml-buildings.parquet/Reg

FileNotFoundError: footprints/global/2022-07-06/ml-buildings.parquet/RegionName=Italy/quadkey=122011000

In [ ]:
gdf = pd.concat(res)

In [ ]:
gdf.to_parquet('../data/ms_buildings_all.pq')

In [48]:
regions_datadir = "/data/uscuni-eurofab/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "ms_ce_region_hulls.parquet"
    )
region_hulls.shape

(474, 1)

In [49]:
prague_hull = region_hulls.loc[[65806]].to_crs(epsg=4326).iloc[0].iloc[0]

In [50]:
intersections = gdf.intersects(prague_hull)

In [51]:
intersections.sum()

np.int64(1259809)

In [52]:
from sidecar import Sidecar
import lonboard

plotting = gdf[intersections]

In [53]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(plotting, opacity=.2)

CPU times: user 2.23 s, sys: 183 ms, total: 2.41 s
Wall time: 2.41 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [54]:
sc = Sidecar(title='tess')

Create a Map object

In [55]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [56]:
with sc:
    display(m)